# Cartpole tests with policy gradient

This notebook contains a simple test for each implemented policy gradient method. In order to test if they function properly, we rely on the [Cartpole](https://gym.openai.com/envs/CartPole-v0/) environment, provided out-of-the-box in OpenAI Gym. As stated in Gym's documentation, the problem is considered "solved" if the agent is able to obtain a mean return of 195 in the last 100 episodes.

## Pre-requisites

The cells down below install and import the necessary libraries to successfully run the notebook examples.

In [1]:
import sys
sys.path.append('../')

In [ ]:
%%capture
!pip install -r ../init/requirements.txt

In [2]:
import numpy as np
import gym

from src import models, policies

%load_ext autoreload
%autoreload 2

## Utilities

The cell down below defines the environment, along with common variables to be used throughout the notebook.

In [3]:
env = gym.make('CartPole-v0')

In [4]:
observation_space_size = 4
action_space_size = 2
hidden_sizes = [32, 32]
epochs = 800
steps_per_epoch = 200
minibatch_size = 100
episodes_mean_return = 100
wandb_config = {
    "api_key": open("../wandb_api_key_file", "r").read().strip(),
    "project": "cpr-appropriation",
    "entity": "wadaboa",
}

## VPG

This section deals with training a Cartpole agent using our custom Vanilla Policy Gradient implementation.

In [6]:
vpg_policy_nn = models.MLP(observation_space_size, hidden_sizes, action_space_size)
vpg_baseline_nn = models.MLP(observation_space_size, hidden_sizes, 1, log_softmax=False)
vpg_policy = policies.VPGPolicy(env, vpg_policy_nn, baseline_nn=vpg_baseline_nn)
vpg_policy.train(
    epochs,
    steps_per_epoch,
    minibatch_size,
    enable_wandb=True,
    wandb_config={**wandb_config, "group": "VPG"},
    episodes_mean_return=episodes_mean_return
)

2021-09-07 17:17:49.160 | DEBUG    | src.models:__init__:56 - Model summary: MLP(
  (mlp): Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=32, out_features=2, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
  )
  (out): LogSoftmax(dim=-1)
)
2021-09-07 17:17:49.162 | DEBUG    | src.models:__init__:56 - Model summary: MLP(
  (mlp): Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=32, out_features=1, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
  )
  (out): Identity()
)


loss,0.03829
mean_return,200.0
_runtime,113
_timestamp,1631027869
_step,728


loss,▆▄▄▃▂▃▂▃▃▆▂▂▃▂▂▂▂▂█▃▂▂▂▁▂▃▂▁▁▁▂▁▁▂▁▂▃▃▃▃
mean_return,▁▂▂▂▄▂▆▅█▇█▇████████████████▇████▅▆▄████
_runtime,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
_timestamp,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████


wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


2021-09-07 17:17:56.889 | INFO     | src.policies:train:123 - Epoch 1 / 800
2021-09-07 17:17:56.889 | INFO     | src.policies:collect_trajectories:221 - Episode 1
2021-09-07 17:17:56.897 | DEBUG    | src.policies:execute_episode:413 - Early stopping, all agents done
2021-09-07 17:17:56.898 | INFO     | src.policies:collect_trajectories:237 - Mean episode return: 34.0
2021-09-07 17:17:56.898 | INFO     | src.policies:collect_trajectories:238 - Last 100 episodes mean return: 34.0
2021-09-07 17:17:56.899 | INFO     | src.policies:collect_trajectories:221 - Episode 2
2021-09-07 17:17:56.904 | DEBUG    | src.policies:execute_episode:413 - Early stopping, all agents done
2021-09-07 17:17:56.904 | INFO     | src.policies:collect_trajectories:237 - Mean episode return: 17.0
2021-09-07 17:17:56.905 | INFO     | src.policies:collect_trajectories:238 - Last 100 episodes mean return: 25.5
2021-09-07 17:17:56.905 | INFO     | src.policies:collect_trajectories:221 - Episode 3
2021-09-07 17:17:56.911

...

2021-09-07 17:18:56.061 | INFO     | src.policies:collect_trajectories:221 - Episode 1672
2021-09-07 17:18:56.202 | DEBUG    | src.policies:execute_episode:413 - Early stopping, all agents done
2021-09-07 17:18:56.203 | INFO     | src.policies:collect_trajectories:237 - Mean episode return: 186.0
2021-09-07 17:18:56.203 | INFO     | src.policies:collect_trajectories:238 - Last 100 episodes mean return: 186.5
2021-09-07 17:18:56.204 | WARNING  | src.policies:train:144 - The actual batch size is 373, instead of 200
2021-09-07 17:18:56.206 | INFO     | src.policies:train:159 - Mini-batch 1 / 3
2021-09-07 17:18:56.209 | INFO     | src.policies:minibatch_update:281 - Losses: {'policy_loss': -0.5090958476066589, 'baseline_loss': 0.8084837198257446, 'total_loss': -0.10485398769378662}
2021-09-07 17:18:56.210 | INFO     | src.policies:minibatch_update:287 - Policy network L2 gradient norm: 0.48964014649391174
2021-09-07 17:18:56.211 | INFO     | src.policies:minibatch_update:291 - Baseline net

2021-09-07 17:18:56.357 | INFO     | src.policies:minibatch_update:305 - Baseline network L2 gradient norm after clipping: 0.49999964237213135
2021-09-07 17:18:56.359 | INFO     | src.policies:train:159 - Mini-batch 2 / 2
2021-09-07 17:18:56.360 | INFO     | src.policies:minibatch_update:281 - Losses: {'policy_loss': -0.5166565179824829, 'baseline_loss': 0.9258697032928467, 'total_loss': -0.05372166633605957}
2021-09-07 17:18:56.360 | INFO     | src.policies:minibatch_update:287 - Policy network L2 gradient norm: 0.14023199677467346
2021-09-07 17:18:56.361 | INFO     | src.policies:minibatch_update:291 - Baseline network L2 gradient norm: 1.3529202938079834
2021-09-07 17:18:56.362 | INFO     | src.policies:minibatch_update:298 - Policy network L2 gradient norm after clipping: 0.14023199677467346
2021-09-07 17:18:56.363 | INFO     | src.policies:minibatch_update:305 - Baseline network L2 gradient norm after clipping: 0.49999964237213135
2021-09-07 17:18:56.365 | INFO     | src.policies:

2021-09-07 17:18:56.546 | INFO     | src.policies:collect_trajectories:221 - Episode 1681
2021-09-07 17:18:56.614 | DEBUG    | src.policies:execute_episode:413 - Early stopping, all agents done
2021-09-07 17:18:56.614 | INFO     | src.policies:collect_trajectories:237 - Mean episode return: 172.0
2021-09-07 17:18:56.615 | INFO     | src.policies:collect_trajectories:238 - Last 100 episodes mean return: 140.0
2021-09-07 17:18:56.615 | WARNING  | src.policies:train:144 - The actual batch size is 280, instead of 200
2021-09-07 17:18:56.618 | INFO     | src.policies:train:159 - Mini-batch 1 / 2
2021-09-07 17:18:56.620 | INFO     | src.policies:minibatch_update:281 - Losses: {'policy_loss': -0.4010089039802551, 'baseline_loss': 0.7162554264068604, 'total_loss': -0.04288119077682495}
2021-09-07 17:18:56.621 | INFO     | src.policies:minibatch_update:287 - Policy network L2 gradient norm: 0.18547064065933228
2021-09-07 17:18:56.622 | INFO     | src.policies:minibatch_update:291 - Baseline net

loss,-0.2117
mean_return,140.0
_runtime,61
_timestamp,1631027936
_step,799


loss,█▆▄▅▅▇▃▅▃▃▃█▃▄▃▅▅▃▂█▅▃▁▆▆▄▇▅▄▄▃▇▄▂▄▂▂▄▄▂
mean_return,▁▁▁▂▂█▆▅▄▇▅▆█▇▆█████▆██▇▆██▆████████▅██▆
_runtime,▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇████
_timestamp,▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


## TRPO

This section deals with training a Cartpole agent using our custom Trust Region Policy Optimization implementation.

In [ ]:
beta = 1.0
kl_target = 0.01

In [ ]:
trpo_policy_nn = models.MLP(observation_space_size, hidden_sizes, action_space_size)
trpo_baseline_nn = models.MLP(observation_space_size, hidden_sizes, 1, log_softmax=False)
trpo_policy = policies.TRPOPolicy(env, trpo_policy_nn, trpo_baseline_nn, beta=beta, kl_target=kl_target)
trpo_policy.train(
    epochs,
    steps_per_epoch,
    minibatch_size,
    enable_wandb=True,
    wandb_config={**wandb_config, "group": "TRPO"},
    episodes_mean_return=episodes_mean_return
)

## PPO

This section deals with training a Cartpole agent using our custom Proximal Policy Optimization implementation.

In [ ]:
alpha = 1.0
beta = 0.01
eps = 0.2

In [ ]:
ppo_policy_nn = models.MLP(observation_space_size, hidden_sizes, action_space_size)
ppo_baseline_nn = models.MLP(observation_space_size, hidden_sizes, 1, log_softmax=False)
ppo_policy = policies.PPOPolicy(env, ppo_policy_nn, ppo_baseline_nn, alpha=alpha, beta=beta, eps=eps)
ppo_policy.train(
    epochs,
    steps_per_epoch,
    minibatch_size,
    enable_wandb=True,
    wandb_config={**wandb_config, "group": "PPO"},
    episodes_mean_return=episodes_mean_return
)